In [1]:
import numpy as np
import cv2
import mediapipe as mp
import os

In [2]:
# Initialize MediaPipe Pose and Drawing modules
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Setup Pose Estimation with MediaPipe
pose = mp_pose.Pose()

def skeletonize(img):
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    height, width, _ = img.shape
    black_image = np.zeros((height, width, 3), dtype=np.uint8)

    if results.pose_landmarks:
    # Draw pose landmarks and skeleton on the black image
        mp_drawing.draw_landmarks(
            black_image, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=1),  # White dots for landmarks
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)   # White lines for connections
        )
    
    return black_image

In [4]:
for i in range(1,5):
    vid_path = "vid-" + str(i) + ".mp4"
    vidcap = cv2.VideoCapture(vid_path)
    success,image = vidcap.read()
    image = cv2.resize(image, (256*image.shape[1]//image.shape[0], 256))
    count = 1
    while success:
        img_dir_path = "../meta-video-pose dataset/data-meta/train_" + str(i) + "_img"
        if not os.path.isdir(img_dir_path):
            os.mkdir(img_dir_path)
        img_write_path = img_dir_path + "/img_"+ str(count) + str(".png")

        cv2.imwrite(img_write_path, image)     # save frame   

        skeleton = skeletonize(image)

        pose_dir_path = "../meta-video-pose dataset/data-meta/train_" + str(i) + "_label"
        if not os.path.isdir(pose_dir_path):
            os.mkdir(pose_dir_path)
        pose_write_path = pose_dir_path + "/label_" + str(count) + str(".png")

        cv2.imwrite(pose_write_path, skeleton)     # save skeleton

        success, image = vidcap.read()
        count += 1
        if count > 3000:
            break

In [9]:
for i in range(1,11):
    img_dir_path = "../video-pose dataset/data-meta/train_" + str(i) + "_img"
    pose_dir_path = "../video-pose dataset/data-meta/train_" + str(i) + "_label"

    # Images file
    img_txt = "train_set/train_" + str(i) + "_img.txt"
    with open(img_txt, "w") as file:
        for filename in os.listdir(img_dir_path):
            path = "train_" + str(i) + "_img/" + filename
            file.write(path + "\n")

    # Images file
    img_txt = "train_set/train_" + str(i) + "_label.txt"
    with open(img_txt, "w") as file:
        for filename in os.listdir(pose_dir_path):
            path = "train_" + str(i) + "_label/" + filename
            file.write(path + "\n")